In [ ]:

import dash
import core
from Scripts.actions import * 
from dash import html,dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output,State
from dash.exceptions import PreventUpdate

# General Settings
# openai.api_type = "azure"
# openai.api_base = "https://chatbotv2.openai.azure.com/"
# openai.api_version = "2022-06-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

divIntro= html.Div(children=[
    dcc.ConfirmDialog(
        id='connect-dialog',
        message="Changing Chatbot Options",
    ),
    dcc.ConfirmDialog(
        id='message-dialog',
    ),
    dbc.Container(fluid=True,className="py-3",
    children=[
        html.H1("MarBot", className="display-3"),
        html.P("This Chatbot is a proof of concept that demonstrates the use of GPT-3 and lower versions as new technologies.\
                It connects to Seeking Alpha and searches for trending news or analysis. The website provides two options for users,\
                Open question' and 'Summary', which allows users to ask questions or create summaries about the articles they choose.",className="lead")
                ],
            )], className="p-3 bg-light rounded-3",
)

divOptions = html.Div(
    dbc.Container( className="display-7", style={'margin-left':'5px'},children=[
        html.H3("Please Select One Option", style={'margin': '10px'}),
        # Chat Option
        html.Div(
            style= {'display':'flex'},
            children=[
                html.H4("ChatBot Mode", style={'margin': '12px'}),
                dbc.RadioItems(id="chatbot-mode",style = {'margin':'15px'}, options=[
                    {"label": "Open Questions", "value": 'OpenChatbot'},
                    {"label": "Smart  Summary", "value": 'Summary'}], inline=True)
                    ]
            ),

        html.Div(style= {'display':'flex'},children=[
            html.H4("Category", style={'margin': '12px'}),
            dbc.RadioItems(id="mode-selected",style = {'margin':'15px'}, options=[
                {"label": "Latest New", "value": 'latest-articles'},
                {"label": "Market News", "value": 'market-news'}], inline=True)]),

        dbc.Select(id="dropdown-article")]),
    className="p-3 bg-light rounded-3",
    style={'border':'1px solid black', 'display': 'inline-block', 'width': '100%', 'vertical-align': 'top', "margin-top":'5px'}
)

divChat = html.Div(id = 'response-container', children=[])

app.layout = html.Div(style={'width': '100%', 'display': 'block', 'padding':'5px'}, children=[
    divIntro,
    html.Hr(style={'margin':'0px'}),
    divOptions,
    divChat])


# Connect2SeekingAlpha
@app.callback(
    Output("connect-dialog", 'displayed'),
    [Input('chatbot-mode', 'value')],
    [State('dropdown-article', 'options')],
)
def DownloadingArticles(value, options):
    if value is None:
        raise PreventUpdate
    else:
        core.TMP_CONTAINER = Connect2Seeking(trigger=True)
        return True

# Adding Article Selection 
@app.callback(
    Output("dropdown-article", 'options'),
    [Input('mode-selected','value')]
)
def UpdateList(mode:str):
    if mode is None:
        raise PreventUpdate
    else:
        tmp_options = []
        for name, _ in core.TMP_CONTAINER[mode].items():
            tmp_options.append({'label': name, 'value': name})
        return tmp_options

# Activate Chat Summary 
@app.callback(
    Output('response-container', 'children'),
    [Input('chatbot-mode','value')]
)
def UpdatingInterface (value:str):
    if value is None:
        raise PreventUpdate
    elif value == core.OPTIONS_MODE[0]:
        interface = html.Div(style={'display':'block'},children=[
            # ChatBot Answer Interface
            html.Div(id='chatbot-response', style={'height': '100px','border':'1px solid black','border-radius':'5px','margin-top':'5px'}),

            # User Interface
            html.Div(id='user-response', style={'display':'flex', 'margin-top':'5px'}, children = [
                dbc.Input(id='message', placeholder='Type your message here', type='text', style={'margin-right':'5px'}),
                html.Button("Submit", id="submit-button", className="btn btn-primary ml-2", n_clicks=0)]
                )
            ]
        )
        return interface
    elif value == core.OPTIONS_MODE[1]:
        interface = html.Div(className='d-flex',style={'margin-top': '10px'}, children=[
            html.Div(id='Summary-container', style={'min-width':'92%','min-height': '100px','border':'1px solid black','border-radius':'5px','margin-right':'5px'}),
            html.Button("Generate Summary", id="summary-button", className="btn btn-primary ml-2", n_clicks=0)])
        return interface

@app.callback(
    [
        Output('message-dialog', 'message'),
        Output('message-dialog', 'displayed'),
        Output('Summary-container','children'),
    ],
    [Input('mode-selected','value'),
    Input('dropdown-article','value'),
    Input('chatbot-mode','value'),
    Input('summary-button', 'n_clicks')
    ]
)
def CreateSummary (Option,Article,Mode,n_clicks):

    if (n_clicks != 0):
        if (n_clicks < 3):
            print(Option,Article,Mode,n_clicks)
            if (Option != None) and (Article != None) and (Mode != None):
                Message, Summary = UserQuestion(Option,Article,Mode, Question = None)
                return Message, True, html.Div(Summary,style={'padding': '10px', 'font-size': '18px'})
            else:
                return  "One or more Options are empty", True, html.Div("Please Make sure Category and Article are Selected",style={'padding': '10px', 'font-size': '18px'})
        else:
            return "Only 3 Summaries, No more Credits", True, html.Div("Only 3 Summaries",style={'padding': '10px', 'font-size': '18px'})
    else:
        raise PreventUpdate


if __name__ == '__main__':
    app.run_server(debug=False)